# data upload

In [16]:
from sqlalchemy import create_engine, types
from datetime import datetime, timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import reliant_scrape as rs
import mysql.connector
import pandas as pd
import numpy as np
import pprint
import yaml
import boto3
import boto
import json
import sys
import os

base = os.path.join('C:\\', 'Users', 'steve', 'Documents', 'reliant-scrape')
data = os.path.join(base, 'data')
pp = pprint.PrettyPrinter(indent = 1)
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader = yaml.FullLoader)
        
with open('db_creds.json', 'r') as f:
    db_creds = json.load(f)
    
print('loaded credentials & imported modules.')

loaded credentials & imported modules.


In [17]:
mydata = rs.mysql_query("""SELECT * FROM reliant_energy_db.daily_use""", db_creds)

Date           datetime64[ns]
Usage (kWh)           float64
Cost ($)              float64
Hi                      int64
Low                     int64
dtype: object

In [20]:
os.chdir(data)
stage = pd.read_csv('daily_usage_11092020.csv')
stage['Date'] = pd.to_datetime(stage['Date'])

In [22]:
result = rs.mysql_query("""SELECT MIN(Date) as min_date, MAX(Date) as max_date, COUNT(*) as count 
                    FROM reliant_energy_db.daily_use""", db_creds)

In [23]:
recent = [d > result.max_date[0] for d in stage.Date]
merge = stage.iloc[recent,:]

In [24]:
merge

,Date,Usage (kWh),Cost ($),Hi,Low
72,2020-11-05 00:00:00,0.5,0.06,78,62
73,2020-11-05 01:00:00,0.5,0.06,78,62
74,2020-11-05 02:00:00,0.6,0.07,78,62
75,2020-11-05 03:00:00,1.0,0.13,78,62
76,2020-11-05 04:00:00,0.5,0.06,78,62
...,...,...,...,...,...
139,2020-11-07 19:00:00,1.1,0.14,81,58
140,2020-11-07 20:00:00,1.0,0.13,81,58
141,2020-11-07 21:00:00,0.7,0.09,81,58
142,2020-11-07 22:00:00,0.6,0.07,81,58


In [25]:
rs.table_upload(merge, 'reliant_energy_db', 'daily_use', db_creds)
print('uploaded data.')

wrote df to sql table.
uploaded data.
